# Special states

<a target="_blank" href="https://colab.research.google.com/github/numqi/numqi/blob/main/docs/application/misc/special_state.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

`numqi.state` implements those speical / interesting states in quantum information.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

try:
    import numqi
except ImportError:
    %pip install numqi
    import numqi


## Werner state

[wiki/werner-state](https://en.wikipedia.org/wiki/Werner_state)

[quantiki/werner-state](https://www.quantiki.org/wiki/werner-state)

[doi-link](http://dx.doi.org/10.1103/PhysRevA.88.032323) Compatible quantum correlations: Extension problems for Werner and isotropic states

**Definition** chosen in `numqi`:

$$ \rho_d(\alpha)=\frac{1}{d^2-d\alpha}I-\frac{\alpha}{d^2-d\alpha}\sum_{ij}\left|ij\right\rangle \left\langle ji\right|,\quad\alpha\in\left[-1,1\right] $$

* SEP range: $\alpha\in\left[-1,\frac{1}{d}\right]$
* (1,k) extension range: $\alpha\in\left[-1,\frac{k+d^{2}-d}{kd+d-1}\right]$
* given a Werner state $\rho\in\mathbb{C}^{d^2\times d^2}$, the corresponding $\alpha=\frac{1-d^2\rho_{00}}{1-d\rho_{00}}$
* PPT can correctly detect the entanglement of Werner state

**Other definition** of Werner state

$$ \rho_{d}\left(a\right)=\frac{d-a}{d\left(d^{2}-1\right)}I+\frac{ad-1}{d\left(d^{2}-1\right)}\sum_{ij}\left|ij\right\rangle \left\langle ji\right|,\quad a\in\left[-1,1\right] $$

Given a Werner state in variable $a$, we can find the corresponding Werner state in variable $\alpha$ by $\alpha=\frac{1-ad}{d-a}$.

In [ ]:
# TODO demo (1,k)-ext boundary

### Relative Entropy of Entanglement (REE)

[doi-link](https://doi.org/10.1103/PhysRevA.64.062307) Entanglement measures under symmetry

$$ E_{R}(\rho(\alpha))=\begin{cases}
S\left(\rho(\alpha)||\rho(1/d)\right) & \alpha>\frac{1}{d}\\
0 & \alpha\leq\frac{1}{d}
\end{cases} $$


In [ ]:
dim = 3
alpha_list = np.linspace(-1, 1, 32)[1:-1] #logm singularity at alpha=1

ree_analytical = np.array([numqi.state.get_Werner_ree(dim, x) for x in alpha_list])
rho_list = np.stack([numqi.state.Werner(dim,x) for x in alpha_list])
ree_ppt = numqi.entangle.get_ppt_ree(rho_list, dim, dim, use_tqdm=True)

fig,ax = plt.subplots()
ax.plot(alpha_list, ree_ppt, label='PPT')
ax.plot(alpha_list, ree_analytical, 'x', label='Analytical')
ax.legend()
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel('REE')


### Entanglement of formation (EoF)

[doi-link](https://doi.org/10.1007/s11704-008-0017-8) Entanglement of formation and concurrence for mixed states

In [ ]:
dim = 3
alpha_list = np.linspace(-1, 1, 32)[1:-1]
# boundary points are singular for numqi.entangle.EntanglementFormationModel

eof_analytical = np.array([numqi.state.get_Werner_eof(dim, x) for x in alpha_list])
eof_opt_list = []
model = numqi.entangle.EntanglementFormationModel(dim, dim, num_term=2*dim*dim)
for alpha in tqdm(alpha_list):
    model.set_density_matrix(numqi.state.Werner(dim, alpha))
    eof_opt_list.append(numqi.optimize.minimize(model, tol=1e-10, print_every_round=0).fun)
eof_opt_list = np.array(eof_opt_list)

fig,ax = plt.subplots()
ax.plot(alpha_list, eof_analytical, 'x', label='Analytical')
ax.plot(alpha_list, eof_opt_list, label='Optimization')
ax.legend()
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel('EoF')


## Isotropic state

[quantiki/isotropic-state](https://www.quantiki.org/wiki/isotropic-state)

[doi-link](http://dx.doi.org/10.1103/PhysRevA.88.032323) Compatible quantum correlations: Extension problems for Werner and isotropic states

Definition chosen in `numqi`:

$$ \rho_d(\alpha)=\frac{1-\alpha}{d^2}I+\frac{\alpha}{d}\sum_{ij}\left|ii\right\rangle \left\langle jj\right|,\quad\alpha\in\left[-\frac{1}{d^{2}-1},1\right] $$

* SEP range: $\alpha\in \left[-\frac{1}{d^2-1},\frac{1}{d+1}\right]$
* (1,k) extension range: $\alpha\in \left[-\frac{1}{d^2-1},\frac{kd+d^2-d-k}{k(d^2-1)}\right]$
* given a Isotropic state $\rho\in\mathbb{C}^{d^2\times d^2}$, the corresponding $\alpha=\frac{d^2\rho_{00}-1}{d-1}$
* PPT can correctly detect the entanglement of Werner state

**Other definition** of Isotropic state

$$ \rho_d\left(a\right)=\frac{d-a}{d\left(d^{2}-1\right)}I+\frac{ad-1}{d\left(d^{2}-1\right)}\sum_{ij}\left|ii\right\rangle \left\langle jj\right|,\quad a\in\left[0,d\right] $$

Given a Isotropic state in variable $a$, we can find the corresponding Isotropic state in variable $\alpha$ by $\alpha=\frac{ad-1}{d^2-1}$.

### Relative Entropy of Entanglement (REE)

(TODO-reference)

$$ E_{R}(\rho(\alpha))=\begin{cases}
S\left(\rho(\alpha)||\rho(\frac{1}{d+1})\right) & \alpha>\frac{1}{d+1}\\
0 & \alpha\leq\frac{1}{d+1}
\end{cases} $$

In [ ]:
dim = 3
alpha_list = np.linspace(-1/(dim*dim-1), 1, 32)[1:-1]

ree_analytical = np.array([numqi.state.get_Isotropic_ree(dim, x) for x in alpha_list])
rho_list = np.stack([numqi.state.Isotropic(dim,x) for x in alpha_list])
ree_ppt = numqi.entangle.get_ppt_ree(rho_list, dim, dim, use_tqdm=True)

fig,ax = plt.subplots()
ax.plot(alpha_list, ree_ppt, label='PPT')
ax.plot(alpha_list, ree_analytical, 'x', label='Analytical')
ax.legend()
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel('REE')


### Entanglement of formation (EoF)

[doi-link](https://doi.org/10.1007/s11704-008-0017-8) Entanglement of formation and concurrence for mixed states

In [ ]:
dim = 3
alpha_list = np.linspace(-1/(dim*dim), 1, 32)[1:-1]
# boundary points are singular for numqi.entangle.EntanglementFormationModel

eof_analytical = np.array([numqi.state.get_Isotropic_eof(dim, x) for x in alpha_list])
eof_opt_list = []
model = numqi.entangle.EntanglementFormationModel(dim, dim, num_term=2*dim*dim)
for alpha in tqdm(alpha_list):
    model.set_density_matrix(numqi.state.Isotropic(dim, alpha))
    eof_opt_list.append(numqi.optimize.minimize(model, tol=1e-10, print_every_round=0).fun)
eof_opt_list = np.array(eof_opt_list)

fig,ax = plt.subplots()
ax.plot(alpha_list, eof_analytical, 'x', label='Analytical')
ax.plot(alpha_list, eof_opt_list, label='Optimization')
ax.legend()
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel('EoF')


## Maximally entangled state

$$ \frac{1}{\sqrt{d}} \sum_{i=1}^{d} |ii\rangle  $$

In [ ]:
numqi.state.maximally_entangled_state(3)

## W state

[wiki/W-state](https://en.wikipedia.org/wiki/W_state)

$$ |W\rangle=\frac{1}{\sqrt{3}}(|001\rangle + |010\rangle + |100\rangle) \in (\mathcal{H}_2)^{\otimes 3} $$

$$ |W_n\rangle=\frac{1}{\sqrt{n}}\sum_{\mathrm{wt}(x)=1} |x\rangle \in (\mathcal{H}_2)^{\otimes n} $$

In [ ]:
numqi.state.W(3)

## Maximally mixed state

$$ \rho_0 = \sum_{i\in[d^2]} \frac{1}{d^2} | ii \rangle\langle ii |.$$

In [ ]:
numqi.state.maximally_mixed_state(2)

## Bell state

$$ |\Phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle) $$

$$ |\Phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle) $$

$$ |\Phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle) $$

$$ |\Phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle) $$

In [ ]:
for i in range(4):
    print(numqi.state.Bell(i))

## GHZ state

$$ |\mathrm{GHZ}\rangle = \frac{1}{\sqrt{2}}(|0\rangle^{\otimes n} + |1\rangle^{\otimes n}) $$

In [ ]:
numqi.state.GHZ(3)